# Fitting a gaussian process model to predict liquid densities

In this notebook we fit Gaussian process models to predict the liquid density resulting from a set of {$\sigma$, $\epsilon$} parameters for R-32.

Points with $\rho_\text{model}$ < 500 kg/m^3 are not used for model fitting. The models perform much worse if that data is included.

# I. Preliminaries

### Imports and path to project
A variety helper functions that are used repeatedly throughout the rest of the notebook.

In [ ]:
import sys
import gpflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

from sklearn import svm

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import silhouette_score


from fffit.utils import (
    shuffle_and_split,
    values_real_to_scaled,
    values_scaled_to_real,
    variances_scaled_to_real,
)

from fffit.plot import(
    plot_model_performance,
    plot_slices_temperature,
    plot_slices_params,
    plot_model_vs_test,
)

from fffit.models import run_gpflow_scipy


sys.path.append('../')

from utils.r32 import R32Constants
from utils.id_new_samples import prepare_df_density

R32 = R32Constants()

liquid_density_threshold = 500 # kg/m^3

iternum=4
cl_shuffle_seed = 58477574
gp_shuffle_seed = 29059757

csv_path = "/scratch365/rdefever/hfcs-fffit/hfcs-fffit/analysis/csv/"
in_csv_names = ["r32-density-iter" + str(i) + "-results.csv" for i in range(1, iternum+1)]

# II. Prepare data

In [ ]:
df_csvs = [pd.read_csv(csv_path + in_csv_name, index_col=0) for in_csv_name in in_csv_names]
df_csv = pd.concat(df_csvs)
df_all, df_liquid, df_vapor = prepare_df_density(
    df_csv, R32, liquid_density_threshold
)

In [ ]:
print(f"{len(df_liquid)} samples remained liquid.")
print(f"{len(df_vapor)} samples went to vapor.")

In [ ]:
plt.scatter(
    values_scaled_to_real(df_all["expt_density"], R32.liq_density_bounds),
    values_scaled_to_real(df_all["md_density"], R32.liq_density_bounds),
)

### Evaluate samples from each iteration

For each iteration:
1. Calculate the MSE between MD and experimental density over all five temperatures for each sample
2. For iterations 2-4; independently sort the first 100 (predicted liquid) and second 100 (predicted vapor) samples by MSE
3. Plot sorted MSE vs. sorted index for both cases

In [ ]:
for iter, df in enumerate(df_csvs):
    df["expt_density"] = df["temperature"].apply(lambda x: R32.expt_liq_density[int(x)])
    df["sq_err"] = (df["density"] - df["expt_density"])**2
    df_mse = df.groupby(list(R32.param_names))["sq_err"].mean().reset_index(name="mse")
    prev_plt = plt.plot(
        range(len(df_mse)),
        df_mse["mse"].sort_values(),
        label=f"Round {iter+1}"
    )
plt.xlabel("Sorted Index")
plt.ylabel("MSE (kg^2/m^6)")
plt.legend()
pass

In [ ]:
for iter, df in enumerate(df_csvs):
    df["expt_density"] = df["temperature"].apply(lambda x: R32.expt_liq_density[int(x)])
    df["sq_err"] = (df["density"] - df["expt_density"])**2
    df_mse = df.groupby(list(R32.param_names))["sq_err"].mean().reset_index(name="mse")
    prev_plt = plt.plot(
        range(len(df_mse)),
        df_mse["mse"].sort_values(),
        label=f"Round {iter+1}"
    )
plt.xlabel("Sorted Index")
plt.ylabel("MSE (kg^2/m^6)")
plt.ylim(-200,1000)
plt.legend()
pass

In [ ]:
param_csv_names = ["r32-density-iter" + str(i) + "-params.csv" for i in range(1, iternum+1)]
result_csv_names = ["r32-density-iter" + str(i) + "-results.csv" for i in range(1, iternum+1)]
df_params = [pd.read_csv(csv_path + param_csv_name, index_col=0) for param_csv_name in param_csv_names]
df_results = [pd.read_csv(csv_path + result_csv_name, index_col=0) for result_csv_name in result_csv_names]
df_mses = []


for iter_ in range(1,iternum+1):
    df_param = df_params[iter_ - 1]
    df_result = df_results[iter_ - 1]

    df_result["expt_density"] = df_result["temperature"].apply(lambda x: R32.expt_liq_density[int(x)])
    df_result["sq_err"] = (df_result["density"] - df_result["expt_density"])**2
    df_mse = df_result.groupby(list(R32.param_names))["sq_err"].mean().reset_index(name="mse")
    
    scaled_param_values = values_real_to_scaled(df_mse[list(R32.param_names)], R32.param_bounds)
    
    param_idxs = []
    for params1 in scaled_param_values:
        for idx, params2 in enumerate(df_param[list(R32.param_names)].values):
            if np.allclose(params1, params2):
                param_idxs.append(idx)
                break
    df_mse["param_idx"] = param_idxs
    df_mses.append(df_mse)

In [ ]:
for i, df_mse in enumerate(df_mses):
    plt.plot(
        df_mse.sort_values("param_idx")["param_idx"],
        df_mse.sort_values("param_idx")["mse"],
        label=f"Round {i+1}"
    )
    plt.xlabel("Parameter set index")
    plt.ylabel("MSE (kg^2/m^6)")
    plt.ylim(0,1100000)
    plt.legend()
    plt.show()
pass

In [ ]:
for i, df_mse in enumerate(df_mses):
    prev_plt = plt.plot(
        range(100),
        df_mse.sort_values("param_idx")[:100].sort_values("mse")["mse"],
        label=f"Round {i+1}"
    )
    plt.plot(
        range(100),
        df_mse.sort_values("param_idx")[100:].sort_values("mse")["mse"],
        '--',
        color=prev_plt[0]._color,
    )
    
plt.xlabel("Sorted Index")
plt.ylabel("MSE (kg^2/m^6)")
plt.legend()
pass

In [ ]:
for i, df_mse in enumerate(df_mses):
    prev_plt = plt.plot(
        range(100),
        df_mse.sort_values("param_idx")[:100].sort_values("mse")["mse"],
        label=f"Round {i+1}"
    )
    plt.plot(
        range(100),
        df_mse.sort_values("param_idx")[100:].sort_values("mse")["mse"],
        '--',
        color=prev_plt[0]._color,
    )
    
plt.xlabel("Sorted Index")
plt.ylabel("MSE (kg^2/m^6)")
plt.ylim(-200,1000)
plt.legend()
pass

### Split into training and test sets (80/20)
The dataframe is shuffled before splitting into train and test sets. We discard the vapor points for training the GP models. We use the `shuffle_seed` option to make the shuffling deterministic.

In [ ]:
model_param_names = list(R32.param_names) + ["temperature"]
property_name = "md_density"
x_train, y_train, x_test, y_test = shuffle_and_split(df_liquid, model_param_names, property_name, shuffle_seed=gp_shuffle_seed)

# III. Kernel comparison

### Create a gaussian process model in GPFlow
Scipy optimizer is used in all cases. First we compare single vs. mulitple variances for the RBF, Matern12, Matern32, and Matern52 kernels. We compare using a single lengthscale vs. multiple lengthscales for each. In all cases multiple lengthscales generally performs better. We then compare all four kernels with multiple lengthscales.

### Observations
* RBF kernel generally performs best (metric: mean square error on test set)
* Full ranking: RBF > Matern52 > Matern32 > Matern12
* Multiple lengthscales often performs slightly better

In [ ]:
all_models = {
    "RBF" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.RBF(lengthscales=1.0)),
    "RBF multiple" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.RBF(lengthscales=np.ones(R32.n_params+1))),
    "Matern12" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern12(lengthscales=1.0)),
    "Matern12 multiple" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern12(lengthscales=np.ones(R32.n_params+1))),
    "Matern32" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern32(lengthscales=1.0)),
    "Matern32 multiple" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern32(lengthscales=np.ones(R32.n_params+1))),
    "Matern52" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern52(lengthscales=1.0)),
    "Matern52 multiple" : run_gpflow_scipy(x_train, y_train, gpflow.kernels.Matern52(lengthscales=np.ones(R32.n_params+1)))
}

### Compare all models (messy plots)

#### Check the predictions of the model at the training points

In [ ]:
plot_model_performance(all_models, x_train, y_train, R32.liq_density_bounds)

#### Check the predictions of the model at the test points

In [ ]:
plot_model_performance(all_models, x_test, y_test, R32.liq_density_bounds)

### Compare selected models

In [ ]:
models_to_compare = ["RBF", "RBF multiple"]
filtered_models = { model : all_models[model] for model in models_to_compare}

#### Check the predictions of the model at the training points

In [ ]:
plot_model_performance(filtered_models, x_train, y_train, R32.liq_density_bounds)

#### Check the predictions of the model at the test points

In [ ]:
plot_model_performance(filtered_models, x_test, y_test, R32.liq_density_bounds)

### Continue forward with only RBF, Matern12, Matern52

In [ ]:
models_to_compare = ["RBF multiple", "Matern12 multiple", "Matern52 multiple"]
models = { model : all_models[model] for model in models_to_compare}

# IV. Model visualization

## Vary temperature

* The linear mean function helps capture the density vs. temperature relationship
* The RBF and Matern52 models are similar while the Matern12 model deviates somewhat
* Model predictions deviate most at $\rho < 600$ kg$/$m$^3$ and when the parameters are at the upper end of their range

In [ ]:
plot_slices_temperature(
    models,
    R32.n_params,
    R32.temperature_bounds,
    R32.liq_density_bounds,
    property_name="Density, kg/m^3"
)

## Vary $\sigma$ and $\epsilon$ at a constant temperature

First we need to define a mapping between the parameter name and the column index.

We also need to select a temperature.

In [ ]:
temperature = 280 # K

for parameter in R32.param_names:
    plot_slices_params(
        models,
        parameter,
        R32.param_names,
        temperature,
        R32.temperature_bounds,
        R32.liq_density_bounds,
        property_name="Density, kg/m^3"
    )

# V. Plot $\rho$ vs. T slices for parameters where at least 1 point was in the training set and 1 point in the test set

This will help us visualize the model variance near and far from the training points. It will also show if the model variance is accurate; i.e., does the MD variance fall within the model mean + variance.

### Plot the model predictions vs. test and training points

In [ ]:
# Loop over test params
for test_params in x_test[:,:R32.n_params]:
    train_points = []
    test_points = []
    # Locate rows where parameter set == test parameter set
    matches = np.unique(np.where((df_liquid.values[:,:R32.n_params] == test_params).all(axis=1))[0])
    # Loop over all matches -- these will be different temperatures
    for match in matches:
        # If the match (including T) is in the test set, then append to test points
        if np.where((df_liquid.values[match,:R32.n_params+1] == x_test[:,:R32.n_params+1]).all(axis=1))[0].shape[0] == 1:
            test_points.append([df_liquid.values[match,R32.n_params],df_liquid.values[match,-3]])
        # Else append to train points
        else:
            train_points.append([df_liquid.values[match,R32.n_params],df_liquid.values[match,-3]])
            
    plot_model_vs_test(
        models,
        test_params,
        np.asarray(train_points),
        np.asarray(test_points),
        R32.temperature_bounds,
        R32.liq_density_bounds,
        property_name="Density, kg/m^3"
    )

## VIII. Can we train a classifier to predict liquid vs. vapor points?
Let's train an SVM.

In [ ]:
param_names = list(R32.param_names) + ["temperature"]
property_name = "is_liquid"
x_train, y_train, x_test, y_test = shuffle_and_split(
 df_all, param_names, property_name, shuffle_seed=cl_shuffle_seed
)
classifier = svm.SVC(kernel="rbf")
classifier.fit(x_train, y_train)
test_score = classifier.score(x_test, y_test)
print(f"Classifer is {test_score*100.0}% accurate on the test set.")

### What if we only train the SVM on the original data?

In [ ]:
df_iter1, df_iter1_liquid, df_iter1_vapor = prepare_df_density(
    df_csvs[0], R32, liquid_density_threshold
)

param_names = list(R32.param_names) + ["temperature"]
property_name = "is_liquid"
x_train_iter1, y_train_iter1, x_test_iter1, y_test_iter1 = shuffle_and_split(
    df_iter1, param_names, property_name, shuffle_seed=cl_shuffle_seed
)
classifier = svm.SVC(kernel="rbf")
classifier.fit(x_train_iter1, y_train_iter1)
test_score = classifier.score(x_test, y_test)
print(f"Classifer is {test_score*100.0}% accurate on the test set.")

### Or only the new data

In [ ]:
df_iter2, df_iter2_liquid, df_iter2_vapor = prepare_df_density(
    df_csvs[1], R32, liquid_density_threshold
)

param_names = list(R32.param_names) + ["temperature"]
property_name = "is_liquid"
x_train_iter2, y_train_iter2, x_test_iter2, y_test_iter2 = shuffle_and_split(
    df_iter2, param_names, property_name, shuffle_seed=cl_shuffle_seed
)
classifier = svm.SVC(kernel="rbf")
classifier.fit(x_train_iter2, y_train_iter2)
test_score = classifier.score(x_test, y_test)
print(f"Classifer is {test_score*100.0}% accurate on the test set.")